In [1]:
import pandas as pd
df=pd.read_csv("Stores.csv")
df

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012
65,66,United States,Wyoming,840.0,1/1/2014


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   StoreKey       67 non-null     int64  
 1   Country        67 non-null     object 
 2   State          67 non-null     object 
 3   Square Meters  66 non-null     float64
 4   Open Date      67 non-null     object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.7+ KB


In [3]:
#  to check any null values that present in the columns of store
df.columns

Index(['StoreKey', 'Country', 'State', 'Square Meters', 'Open Date'], dtype='object')

In [4]:
nullvalues=df['Square Meters'].isna().sum()
print(nullvalues)

1


In [5]:
nullrows=df.loc[df['Square Meters'].isna()]
print(nullrows)

    StoreKey Country   State  Square Meters Open Date
66         0  Online  Online            NaN  1/1/2010


In [6]:
# there is no correct value for any of the columns in the row and Nan values present in the rows.so i drop the row
df.dropna(how='any',subset=['Square Meters'],inplace=True)

In [7]:
df

,StoreKey,Country,State,Square Meters,Open Date
0,1,Australia,Australian Capital Territory,595.0,1/1/2008
1,2,Australia,Northern Territory,665.0,1/12/2008
2,3,Australia,South Australia,2000.0,1/7/2012
3,4,Australia,Tasmania,2000.0,1/1/2010
4,5,Australia,Victoria,2000.0,12/9/2015
...,...,...,...,...,...
61,62,United States,South Dakota,1120.0,6/3/2018
62,63,United States,Utah,2000.0,3/6/2008
63,64,United States,Washington DC,1330.0,1/1/2010
64,65,United States,West Virginia,1785.0,1/1/2012


In [8]:
# open date is in object ,so i am going to change this in date format
df['Open Date']=pd.to_datetime(df['Open Date'])

In [10]:
# TO ESTABLISH PSYCOPG2 CONNECTION:
import psycopg2

connection = psycopg2.connect(
    host="localhost",
    database="Dataspark",
    user="postgres",
    password="12345678"
)

cursor = connection.cursor()


cursor.execute("SELECT version()")

version = cursor.fetchone()

print("You are connected to - ", version)

connection.close()


You are connected to -  ('PostgreSQL 15.8, compiled by Visual C++ build 1940, 64-bit',)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 66 entries, 0 to 65
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   StoreKey       66 non-null     int64         
 1   Country        66 non-null     object        
 2   State          66 non-null     object        
 3   Square Meters  66 non-null     float64       
 4   Open Date      66 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 3.1+ KB


In [11]:
#To create a database
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

connection = psycopg2.connect(
    host="localhost",
    port="5432",
    database="Dataspark",
    user="postgres",
    password="12345678"
)

connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

writer = connection.cursor()

#writer.execute("create database hello")

writer.execute("""create table store("StoreKey" integer,"Country" text,"State" text,"Square Meters" real,"Open Date" date)""")

writer.close()
connection.close()

In [12]:
import pandas as pd
from sqlalchemy import create_engine

# pip install pymysql
# import pymysql


# Replace with your connection details
host="localhost"
port="5432"
database="Dataspark"
user="postgres"
password="12345678"

# Create the connection string (URL format) -> postgres
engine_string = f"postgresql://{"postgres"}:{12345678}@{"localhost"}:{5432}/{"Dataspark"}"

# Create the connection string (URL format) -> mysql
# engine_string = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}"

# Create the SQLAlchemy engine
engine = create_engine(engine_string)

# Sample DataFrame (replace with your actual data)

df = pd.DataFrame(df)

# Table name where you want to push the data
table_name = "store"

# Push the DataFrame to the PostgreSQL table
df.to_sql(table_name, engine,if_exists='replace', index=False)  #['fail', 'replace', 'append']

print("Data successfully pushed to PostgreSQL table!")

Data successfully pushed to PostgreSQL table!
